# Week 1 Examples — Time, Clocks, and Latency

This notebook demonstrates key Week 1 concepts: measuring time, clock drift, and latency distributions. Use Python 3.9+.

---

## Measuring time reliably
Use `time.perf_counter()` for precise elapsed time measurements. Avoid wall-clock for durations.

Key ideas:
- Prefer monotonic timers for durations
- Warm up before measuring
- Report distribution (median, p95, p99)


In [ ]:
import time, statistics, math, random


def measure_loop(n=10000):
    start = time.perf_counter()
    s = 0
    for i in range(n):
        s += i
    end = time.perf_counter()
    elapsed = end - start
    return elapsed, s

# Warm-up
_ = measure_loop(50000)

runs = [measure_loop(200000)[0] for _ in range(15)]
print({
    'median_ms': statistics.median(runs)*1e3,
    'p95_ms': sorted(runs)[int(0.95*(len(runs)-1))]*1e3,
    'p99_ms': sorted(runs)[int(0.99*(len(runs)-1))]*1e3,
    'min_ms': min(runs)*1e3,
    'max_ms': max(runs)*1e3,
})


## Clock drift simulation
Two clocks drift by ±50 ppm (parts per million). We simulate 1 hour and compute skew over time.

Interpretation:
- rate = 1 + ppm/1e6
- skew(t) = t*(rate_a - rate_b)
- 50 ppm ≈ 0.05 ms per second, ≈ 180 ms per hour.


In [ ]:
def simulate_drift(seconds=3600, ppm_a=30, ppm_b=-40):
    # ppm -> fractional rate error
    rate_a = 1 + ppm_a/1_000_000
    rate_b = 1 + ppm_b/1_000_000
    skew = []
    a = 0.0
    b = 0.0
    dt = 1.0
    for _ in range(int(seconds)):
        a += dt*rate_a
        b += dt*rate_b
        skew.append(a-b)
    return skew

skew = simulate_drift()
print(f'Skew after 1 hour: {skew[-1]*1e3:.2f} ms')
print(f'Max |skew|: {max(abs(x) for x in skew)*1e3:.2f} ms')


## Synthetic network latency with heavy tail
We generate a base latency plus occasional slowdowns. Compute p50/p95/p99 and show why tails matter.

Scenarios that create tails:
- GC pauses; CPU contention
- Shared network links; intermittent retransmissions
- Storage hiccups (write amplification)


In [ ]:
def heavy_tail_latencies(n=5000, base_ms=2.0, jitter_ms=0.5, slow_p=0.02, slow_ms=50):
    xs = []
    for _ in range(n):
        x = random.gauss(base_ms, jitter_ms)
        if random.random() < slow_p:
            x += slow_ms * random.random()
        xs.append(max(0, x))
    return xs


def percentile(xs, p):
    ys = sorted(xs)
    k = int((p/100.0)*(len(ys)-1))
    return ys[k]

xs = heavy_tail_latencies()
print('p50=', percentile(xs, 50))
print('p95=', percentile(xs, 95))
print('p99=', percentile(xs, 99))
print('max=', max(xs))


### Optional plot
If available, plot with matplotlib:
```python
import matplotlib.pyplot as plt
plt.hist(xs, bins=100, color='#93c5fd')
for p in [50,95,99]:
    v = percentile(xs, p)
    plt.axvline(v, color='red', linestyle='--')
plt.title('Latency distribution with heavy tail')
plt.xlabel('ms'); plt.ylabel('count'); plt.show()
```
